<a href="https://colab.research.google.com/github/BYU-Hydroinformatics/ee-wq-lasso/blob/main/notebook_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Authors: Anna Cardall, Kel Markert & Gus Williams*, Brigham Young University

🚨 Please use the following citation for this notebook:

Cardall, A. C., Hales, R. C., Williams, G.P., Tanner, K. B., Markert, Kel N. (2023). LASSO (L1) Regularization for Development of Sparse Remtoe Sensing Models with Applications to Optically Complex Waters using GEE Tools. Remote Sensing.

</br>

##**Introduction**
Remote sensing tools can provide spatial and temporal resolution for monitoring water quality in reservoirs and large rivers that are not available from traditional *in situ* measurements. The retrieval of water quality parameters from remote sensing systems relies on the optical properties (transmittance, absorption and scattering) of water and the dissolved and suspended constituents in the water. Suspended solids are responsible for most of the scattering in an aquatic system, whereas chlorophyll-*a* (chl-*a*) and colored dissolved matter are mainly responsible for absorption ([Myint and Walker, 2002](https://doi.org/10.1080/01431160110104700
)). There is a wide body of research on assessing water quality analytical optical modeling using in situ inherent optical properties ([Cox et al., 2009](https://doi.org/10.1080/07438149809354347)). Methods used to relate in situ data to the satellite observations through statistical relationships include simple linear regression, non-linear regressions, principal component analysis, and neural networks. </br>

This notebook generates the data pairs of in-situ measurements and satellite data needed to create such models. It takes a CSV file of *in-situ* water quality data, dates, and locations of each measurement and exports and CSV file containing the *in-situ* measurement information and the band values of a near-coincident satellite image at the sample location along with the time offset between the in-situ measurement and the satellite collection. We use Landsat data in this notebook, but the code can be modified to use other satellites.

## **Instructions**

Obtaining the dataset requires fives steps:
1. Setup Google Earth Engine (GEE)
2. Prepare and import the in-situ dataset 
3. Make some selections for time zone and resolution
4. Click 'Run' on a group of cells

Each step has its own heading and detailed instructions below.</br></br>


This tool does not require any programming, but some understanding of the code might be helpful if you would like to make adjustments or additions. If you are new to Python programming, go through [this guide](https://docs.python.org/3/tutorial/index.html).  Review the [Earth Engine documentation](https://developers.google.com/earth-engine/guides) and the [Python Guide](https://developers.google.com/earth-engine/guides/python_install).

A few links that are helpful:

*   [Earth Engine Documentation](https://developers.google.com/earth-engine)
*   [Earth Engine API Reference](https://developers.google.com/earth-engine/apidocs)
* [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets)
* [Earth Engine Best Practices](https://developers.google.com/earth-engine/guides/best_practices)
* [Tethys Portal](https://tethys-staging.byu.edu/apps/lake/data/)

## **1. Setup Google Earth Engine (GEE)**

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. We will also install a package for visualizing results. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity. 

You will need a Google Earth Engine account to run this notebook. 

In [ ]:
# install geemap package for visualizing ee results
!pip install geemap &> install.log

In [ ]:
# import ee api, geemap package, and Jupyter Widgets package
import ee
import math
import geemap
import numpy as np
import pandas as pd
from geemap import colormaps as cmaps
import ipywidgets as widgets
from IPython.display import display
%config InteractiveShell.ast_node_interactivity = 'all'

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=1vPc0JkotksHxtA40Pd5h9t0NpEj0NPun2m6bKq9N2E&tc=QQQZVjtO90J1sVtUNQNqRczhs5Mr9I1-lqPx53l2H0U&cc=_xX5yzdsIpTzxNpyWUWmnsyPNR-zUGnQIvzD8ZWQ2Go

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh4kgDDFNriKKzgFBQQdYB0X3fIe3xFVpUR5e5X34VHBwBPb9Fdkwvk

Successfully saved authorization token.


## **2. Load in the dataset of in-situ samples**

### **Prepare CSV file**
Your file should have the columns shown in the following table. The column headins should match exactly. You may alter either your file or the code for the notebook to process your data, the former being the easier option. But warned, there might be some debugging involved here! 

See this [link](http://joda-time.sourceforge.net/apidocs/org/joda/time/format/DateTimeFormat.html) for help with formatting data representing the dates. See this [link](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones) for a list of allowable time zone strings. You can use the strings under "TZ database name" or "Time zone abbreviation" columns.

While the below table has the column names listed vertically, in the spreadsheet they should be the horizontal column names and on row 1. 

</br>

| Column Name       | Format     | Example    
| ----------------- | ---------- | ----------- 
| date              | MM/dd/YYYY | 7/11/1989 
| time              | HH:mm:ss a | 9:30:00 AM
| time zone         |            | MST
| latitude          |            | 40.26828
| longitude         |            | -111.82993
| parameter name    |            | chlorophyll-a
| measurement value |            | 31.8
| unit              |            | ug/L

</br>

### **Clean your dataset**
This notebook removes any rows not containing a date, time, latitude, longitude, or measurement value. However, before uploading, you should perform your own data data cleaning to remove such rows or any other rows otherwise unsuitable for your research purposes. For example, you may want to remove rows with unreasonably high, low, or physically impossible measurement values, data outside of your period of interest, or duplicates.

</br>

### **Import CSV file into GEE as an asset**
To bring a file into GEE, you must first upload it as an asset. To do this, go to https://code.earthengine.google.com/, click the "Assets" tab on the left, click the red 'New' button, and select "CSV file (.csv)" under the "Table Upload" header. 
*  If the columns for latitude and longitude are not "latitude" and "latitude" in your .csv file, either change the header or specify the headers under the "Advanced Options" heading in the upload menu. This allows GEE to construct point geometries for the samples. </br>
*  Investigate any error messages that appear if uploading the file fails, and look under the "Advanced Options" heading in case you need to make any additional specifications.

Once it's uploaded, click on the file in the list assets and copy its Table ID (aka Asset ID) and paste it below. It should look something like 'users/your_username/file_name' or 'projects/my-cool-project/assets/file_name' (without the quotes).

The below "file_path" value is an example. Yours will be different. 

In [ ]:
#@markdown **Enter a file path**

# Colab form field for a string
file_path = "projects/byu-models/assets/AWQMS_turbidity" #@param {type:"string"}

# pull in the dataset 
# and delete rows with NANs in important columns
dataset = (
    ee.FeatureCollection(file_path)
    .filter(ee.Filter.neq('date', None))
    .filter(ee.Filter.neq('time', None))
    .filter(ee.Filter.neq('latitude', None))
    .filter(ee.Filter.neq('longitude', None))
    .filter(ee.Filter.neq('measurement value', None))
)

# print out the number of samples
print(f"Total number of samples: {dataset.size().getInfo()}"+"\n")

# Display a map of the in situ samples
Map = geemap.Map()

Map.centerObject(dataset, 10); 

Map.addLayer(dataset,{},"In-Situ Samples")

Map.addLayerControl()

Map

Total number of samples: 586



Map(center=[40.237799209199274, -111.80487311788401], controls=(WidgetControl(options=['position', 'transparen…

##**3. Make selections**

In [ ]:
#@markdown Select a resolution. "30" uses a single 30-m  pixel
#@markdown containing the coordinate of the in-situ sample. Select "90" 
#@markdown if you would like information of a 90-m pixel, an aggregation of a 
#@markdown 3x3 pixel grid (see [this page](https://developers.google.com/earth-engine/guides/scale) 
#@markdown for more information on *scale* in GEE).
resolution = "30" #@param [30,90]
resolution = float(resolution)

#@markdown Enter a time zone to format the satellite image date by. 
#@markdown See this [link](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones) 
#@markdown for a list of allowable time zone strings. You can use the strings under 
#@markdown "TZ database name" or "Time zone abbreviation" columns.
#@markdown Defaults to UTC. This is the time zone of the samples in the CSV file. 
time_zone = "America/Denver" #@param {type:"string"}

## **4. Run these cells!**

### **Prepare the in-situ dataset**

In [ ]:
# format date information for samples
def format_sample_date(feature):

    # extract day, time, and time zone
    sample_day = ee.String(feature.get('date'))
    sample_time = ee.String(feature.get('time'))
    sample_timeZone = ee.String(feature.get('time zone'))

    # combine into a single string
    sample_date = sample_day.cat(' ').cat(sample_time).cat(' ').cat(sample_timeZone)

    # convert date to milliseconds since 1970
    sample_timestamp = ee.Date.parse('MM/dd/YYYY HH:mm:ss a z', sample_date).millis()

    return feature.set("sample date", sample_date,'sample timestamp (ms)', sample_timestamp)

dataset = dataset.map(format_sample_date)

### **Prepare satellite data**

#### **Scaling factor function**

In [ ]:
# function to apply Collection 2 scaling factors
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  image = image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)
  return image

#### **Land & quality mask function**

In [ ]:
# load historical suface water occurrence information to constain water masks
water_occurrence = ee.Image("JRC/GSW1_3/GlobalSurfaceWater").select("occurrence")
water_constrain = water_occurrence.gt(50)

# QA mask function
def qa_mask(image):
    # Bits 3, 4, and 5 are cloud, cloud shadow, and snow, respectively.
    cloudsBitMask = (1 << 3);
    cloudShadowBitMask = (1 << 4);
    snowBitMask = (1 << 5);

    #Get the pixel QA band.
    qa = image.select('QA_PIXEL');

    # apply the bit shift and get binary image of different QA flags
    cloud_qa = qa.bitwiseAnd(cloudsBitMask).eq(0)
    cloud_shadow_qa = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
    snow_qa = qa.bitwiseAnd(snowBitMask).eq(0)

    # get water mask info!
    waterBitMask = (1 << 7)
    # and constrain to where we know there is water
    water_qa = qa.bitwiseAnd(waterBitMask).updateMask(water_constrain)

    # combine qa mask layers to one final mask
    mask = cloud_shadow_qa.And(snow_qa).And(cloud_qa).And(water_qa)

    # apply mask and return orignal image
    return image.updateMask(mask);

#### **Band renaming function**

In [ ]:
# function to rename Landsat 5 & 7 bands
def L57_BandRenamer(image):
  image = image.select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7','ST_B6'],
                       ['blue','green','red','NIR','SWIR1','SWIR2','SurfTempK'])
  return image

# function to rename Landsat 8 & 9 bands. We do not select UltraBlue
def L89_BandRenamer(image):
  image = image.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','ST_B10'],
                       ['blue','green','red','NIR','SWIR1','SWIR2','SurfTempK'])
  return image

#### **Load in Landsat data**

In [ ]:
# Collect all Tier 1, Landsat Collection 2 SR images of the area for each satellite
L5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filterBounds(dataset)
L7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterBounds(dataset)
L8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(dataset)
L9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterBounds(dataset)

#### **Scale, quality mask, and rename**

In [ ]:
# scale, quality mask, and rename all satellite data
L5_prepared = L5.map(applyScaleFactors).map(qa_mask).map(L57_BandRenamer)
L7_prepared = L7.map(applyScaleFactors).map(qa_mask).map(L57_BandRenamer)
L8_prepared = L8.map(applyScaleFactors).map(qa_mask).map(L89_BandRenamer)
L9_prepared = L9.map(applyScaleFactors).map(qa_mask).map(L89_BandRenamer)

# Merges the prepared satellite collections 
# and sorts the new collection by date
satellite_data = L5_prepared.merge(L7_prepared).merge(L8_prepared).merge(L9_prepared)
satellite_data = satellite_data.sort('system:time_start')

### **Find near-coincident data**

#### **Get a list of sample dates**

In [ ]:
dates = (
    dataset
    .aggregate_array('sample timestamp (ms)')
    .distinct()
)

# get how many dates we have to loop through
n = dates.size().getInfo()
print(f"Number of collection dates: {n}")

Number of collection dates: 585


#### **Loop to find Near Coincident data from satellite**

In [ ]:
import sys
sys.setrecursionlimit(10000)

In [ ]:
# create an empty featurecollection to append samples to
data_pairs = ee.FeatureCollection([])

# start looping over dates
for i in range(n):
    sample_date = ee.Date(dates.get(i))

    tolerance = 5 # days

    # get time bounds to filter imagery
    t1 = sample_date.advance(-tolerance,"day")
    t2 = sample_date.advance(tolerance,"day")

    # get the samples from the date of interest
    samples_date = dataset.filter(ee.Filter.eq('sample timestamp (ms)',sample_date.millis()))

    # filter imagery for date and mosaic
    sample_img = (
        satellite_data
        .filterDate(t1,t2)
        .map(lambda x: x.addBands(x.metadata('system:time_start','image_timestamp (ms)')))
        .mosaic()
        .addBands(ee.Image.pixelLonLat())
    )

    # sample pixels using the sample points
    spectra_samples = sample_img.sampleRegions(
        collection = samples_date,
        scale = resolution,
        tileScale = 4, 
        geometries = True
    )

    # append samples from date to larger collection
    data_pairs = data_pairs.merge(spectra_samples)

# filter by a band to make sure we only have samples from valid observations
# and remove nans
data_pairs = data_pairs.filter(ee.Filter.neq("blue", None))

### **Prepare export**

In [ ]:
# add pixel merging, time window, and image date columns
def addProperties(feature):
    # adding column indicating if 3x3 pixel grid was used for image data
    new_feature = feature.set('resolution', resolution)

    # computing the # of hours between samples and the satellite image
    sample_timestamp = ee.Number(feature.get('sample timestamp (ms)'))
    image_timestamp = ee.Number(feature.get('image_timestamp (ms)'))
    # subtracting the ms timestamps, converting to hours, and taking absolute value
    time_window = ee.Number.expression('(s-i)/3600000', {'s':sample_timestamp,'i': image_timestamp}).abs()
    new_feature = new_feature.set('time window', time_window)

    # adding an image date column with a readable date
    image_date = ee.Date(image_timestamp).format('MM/dd/YYYY HH:mm:ss a z', time_zone)
    new_feature = new_feature.set('image date',image_date)

    return new_feature

dataPairs = data_pairs.map(addProperties)

In [ ]:
# sort by ascending date
dataPairs = dataPairs.sort('sample timestamp (ms)')

In [ ]:
# list of columns to keep in the .csv export, in order
properties = ['parameter name','measurement value','unit','latitude','longitude','sample date','time window','image date','resolution','blue','green','red','NIR','SWIR1','SWIR2','SurfTempK']

In [ ]:
def export():
  drive_task = ee.batch.Export.table.toDrive(
      collection = dataPairs,
      description = file_name,
      folder = folder_name,
      selectors = properties # columns
  )
  drive_task.start()

## **Export**

Run the following when you're ready to export the CSV file. You can check on the status of the export in the [Earth Engine Task Manager](https://code.earthengine.google.com/tasks).

In [ ]:
#@markdown Enter an output file name. 
file_name = "dataPairs_v01" #@param {type:"string"}

#@markdown Enter an output folder name. This is a folder on your Google Drive. 
#@markdown Google Colaboratory does not support filepaths currently, so the notebook will
#@markdown output to the most recently used folder of the entered folder name. 
folder_name = "data_pairs" #@param {type:"string"}

In [ ]:
export()